In [276]:
import pandas as pd
import numpy as np
import warnings
import re
warnings.filterwarnings("ignore")

from xgboost import XGBRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import train_test_split

In [277]:
#Data import
df=pd.read_csv("data/instagram_reach_fin.csv",index_col=0)

In [278]:
df.head()

,S.No,USERNAME,Caption,Followers,Hashtags,Time since posted,Likes
0,1,mikequindazzi,Who are #DataScientist and what do they do? >>...,1600,#MachineLearning #AI #DataAnalytics #DataScien...,11,139
1,2,drgorillapaints,We all know where it’s going. We just have to ...,880,#deck .#mac #macintosh#sayhello #apple #steve...,2,23
2,3,aitrading_official,Alexander Barinov: 4 years as CFO in multinati...,255,#whoiswho #aitrading #ai #aitradingteam#instat...,2,25
3,4,opensourcedworkplace,sfad,340,#iot #cre#workplace #CDO #bigdata #technology#...,3,49
4,5,crea.vision,Ever missed a call while your phone was chargi...,304,#instamachinelearning #instabigdata#instamarke...,3,30


In [279]:
#irrelevant columns
df = df.drop(['S.No',"USERNAME"], axis=1)

In [280]:
#Preprocessing and feature engineering
vectorizer = TfidfVectorizer(stop_words='english')
caption_features = vectorizer.fit_transform(df['Caption'])
dff = caption_features.toarray()
dfx = pd.DataFrame(dff)

In [281]:
#Combining the data
features = df[['Followers', 'Time since posted']]
features = features.drop_duplicates().reset_index(drop=True)
combined_df = pd.concat([features,dfx],axis=1)

In [282]:
#Dealing with column names as str
cols =[i for i in combined_df]
hx = list(map((lambda x:str(x) if type(x)!="str" else None),cols))
di = {}
for i,j in zip(cols,hx):
    di[i] = j
combined_df=combined_df.rename(di,axis=1)

In [283]:
# Split the data 
X_train, X_test, y_train_l, y_test_l = train_test_split(combined_df, df['Likes'], test_size=0.2, random_state=55)
X_train, X_test, y_train_t, y_test_t = train_test_split(combined_df, df['Time since posted'], test_size=0.2, random_state=55)


In [284]:
# Model training
model_l = XGBRegressor()
model_l.fit(X_train, y_train_l,verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [285]:
model_t = XGBRegressor()
model_t.fit(X_train, y_train_t)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [286]:
# Model evaluation
y_pred_l = model_l.predict(X_test)
mse_l = mean_squared_error(y_test_l, y_pred_l)

y_pred_t = model_t.predict(X_test)
mse_t = mean_squared_error(y_test_t, y_pred_t)


print(f"Mean Squared Error (Likes): {mse_l}")
print(f"Mean Squared Error (Time since posted): {mse_t}")


Mean Squared Error (Likes): 5345.104733762956
Mean Squared Error (Time since posted): 0.025016278156731887
